## CS 584 Machine Learning : Homework Project: Crime Prediction
---
### Team Members:
 * Jadhav Aditya CWID
 * Sumedha Gupta CWID


In [649]:
import pandas as pd # panda's nickname is pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from  sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import binarize
import matplotlib.pyplot as plot
from sklearn import linear_model
from IPython.display import Image 
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn import tree
import sklearn.preprocessing as prep
import pydotplus 

from sklearn.ensemble import GradientBoostingClassifier


#### Load the communities-crime-clean.csv file into a dataframe object


In [672]:
communities_crime_df = pd.read_csv('communities-crime-clean.csv')

# Sanity test we have good data
#communities_crime_df.head()
import nltk
print(nltk.__version__)

3.2.2


### Q1. Decision Trees
---
#### a) Create a new field “highCrime” which is true if the crime rate per capita (ViolentCrimesPerPop) is greater than 0.1, and false otherwise. What are the percentage of positive and negative instances in the dataset?

In [651]:
def setHighCrime(df):
    if df['ViolentCrimesPerPop'] > 0.1:
        return True
    else:
        return False

communities_crime_df['highCrime'] = communities_crime_df.apply(setHighCrime, axis=1)
communities_crime_df.groupby('highCrime').size() * 100 / len(communities_crime_df)



highCrime
False    37.280482
True     62.719518
dtype: float64

#### b) Use DecisionTreeClassifier to learn a decision tree to predict highCrime on the entire dataset  
##### i. What are the training accuracy, precision, and recall for this tree?

In [652]:
df = communities_crime_df.drop('ViolentCrimesPerPop', axis=1).drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1).drop('highCrime', axis=1)


features = list(df.columns)

y = communities_crime_df["highCrime"]
X = df
dt = DecisionTreeClassifier()

dt.fit(X,y)
predicted = dt.predict(df[features])
#communities_crime_df['Predicted_HighCrime'] = pd.Series(p, index=communities_crime_df.index)
#dot_data = tree.export_graphviz(dt, out_file=None)
#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png()) 
#Thus in binary classification, the count of true negatives is C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}.
#metrics.confusion_matrix(communities_crime_df['highCrime'], predicted)
recall_score = metrics.recall_score(communities_crime_df['highCrime'], predicted)
precision_score = metrics.precision_score(communities_crime_df['highCrime'], predicted)
accuracy_score = metrics.accuracy_score(communities_crime_df['highCrime'], predicted)

print("Training Accuracy = {} Precision = {} Recall = {}".format(accuracy_score,precision_score,recall_score))

Training Accuracy = 1.0 Precision = 1.0 Recall = 1.0


##### ii. What are the main features used for classification? Can you explain why they make sense (or not)?
TODO


#### c. Now apply cross-validation (cross_val_score) to do 10-fold cross-validation to estimate the out-of-training accuracy of decision tree learning for this task.  
##### i. What are the 10-fold cross-validation accuracy, precision, and recall?

In [653]:
scores = cross_val_score(dt, X, y, cv=10)

precision= cross_val_score(dt, X, y, cv=10, scoring='precision')
recal = cross_val_score(dt, X, y, cv=10, scoring='recall')
print("Results after applying 10-fold cross-validation")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
print("Recall", np.mean(recal))
print("Precision", np.mean(precision))


Results after applying 10-fold cross-validation
Accuracy: 0.73 (+/- 0.05)
Recall 0.7728
Precision 0.78313350665


##### ii. Why are they different from the results in the previous test?
TODO

### 2. Linear Classification
---
#### a. Use GaussianNB to learn a Naive Bayes classifier to predict highCrime.  
##### i. What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [654]:
clf = GaussianNB()

clf.fit(X, y)
predicted = clf.predict(communities_crime_df[features])
#communities_crime_df['abc'] = pd.Series(p, index=communities_crime_df.index)
#communities_crime_df
#communities_crime_df.head()
#for i in communities_crime_df['abc']:
#for index, row in communities_crime_df.iterrows():    
#    print(type(row['abc']))
#confusion_matrix(communities_crime_df['highCrime_Boolean'], communities_crime_df['abc'])
#recall_score(communities_crime_df['highCrime'], communities_crime_df['abc'])
#recall_score(a['highCrime_Boolean'], a['abc'])
#communities_crime_df.head()
#accuracy_score1 = metrics.recall_score(communities_crime_df['highCrime'], predicted)
scores = cross_val_score(clf, X, y, cv=10)
precision= cross_val_score(clf, X, y, cv=10, scoring='precision')
recal = cross_val_score(clf, X, y, cv=10, scoring='recall')
print(np.mean(recal))
print(np.mean(precision))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))



0.692
0.911799814828
Accuracy: 0.76 (+/- 0.05)


What are the 10 most predictive features? This can be
measured by the normalized absolute difference of means for
the feature between the two classes:
|𝜇𝑇 − 𝜇𝐹|/𝜎𝑇 + 𝜎𝐹
The larger this different, the more predictive the feature. Why
do these make sense (or not)?

In [655]:
dfPredictiveFeature = communities_crime_df.drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1)

dictPredFeature = {}
for column in dfPredictiveFeature:
    meanTrue = dfPredictiveFeature[dfPredictiveFeature['highCrime'] == 1][column].mean()
    meanFalse = dfPredictiveFeature[dfPredictiveFeature['highCrime'] == 0][column].mean()
    varTrue = dfPredictiveFeature[dfPredictiveFeature['highCrime'] == 1][column].var()
    varFalse = dfPredictiveFeature[dfPredictiveFeature['highCrime'] == 0][column].var()
    if(column != 'highCrime'):
        predScore = abs((meanTrue - meanFalse))/(varFalse+varTrue)
        dictPredFeature[column] = predScore
MostPredFeat = sorted(dictPredFeature.items(), key=lambda x: x[1])[-11:-1]
for i in MostPredFeat:
    print(i)
dfPredictiveFeature.head()

('racePctWhite', 3.4402715072040073)
('PctIlleg', 3.466925445469923)
('PctYoungKids2Par', 3.644782922632504)
('MalePctDivorce', 3.9465337329938555)
('PctTeen2Par', 3.9954561774196335)
('TotalPctDiv', 4.3812734118491115)
('pctWInvInc', 4.390239171514135)
('PctFam2Par', 4.5894616248835165)
('FemalePctDiv', 4.751737777973133)
('PctKids2Par', 4.996478183496772)


,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop,highCrime
0,0.01,0.61,0.21,0.83,0.02,0.01,0.41,0.49,0.26,0.21,...,0.70,0.40,0.34,0.57,0.05,0.06,0.01,0.0,0.06,False
1,0.01,0.41,0.55,0.57,0.01,0.00,0.47,0.45,0.31,0.57,...,0.93,0.66,0.82,0.84,0.11,0.03,0.01,0.0,0.14,True
2,0.03,0.34,0.86,0.30,0.04,0.01,0.41,0.42,0.27,0.59,...,0.77,0.59,0.70,0.64,0.06,0.11,0.04,0.0,1.00,True
3,0.01,0.38,0.35,0.71,0.04,0.01,0.39,0.46,0.31,0.49,...,0.78,0.56,0.67,0.71,0.09,0.05,0.00,0.0,0.23,True
4,0.04,0.37,0.32,0.70,0.21,0.02,1.00,1.00,1.00,0.14,...,0.49,0.12,0.00,0.15,0.09,0.09,0.01,0.0,0.15,True


#### b. Use LinearSVC to learn a linear Support Vector Machine model to predict highCrime.
##### i. What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [656]:
linearsvmclf = svm.SVC(kernel='linear', C =1.0)
accuracy_svm = cross_val_score(linearsvmclf, X, y, cv=10, scoring='accuracy').mean()
precision_svm = cross_val_score(linearsvmclf, X, y, cv=10, scoring='precision').mean()
recall_svm = cross_val_score(linearsvmclf, X, y, cv=10, scoring='recall').mean()
print ('Accuracy is', accuracy_svm)
print ('Precision is', precision_svm)
print ('Recall is', recall_svm)

Accuracy is 0.804753768844
Precision is 0.855179372789
Recall is 0.8376


#### ii. What are the 10 most predictive features? This can be measured by the absolute feature weights in the model. Why do these make sense (or not)?

In [657]:
linearsvmclf.fit(X,y)
#print(linearsvmclf.coef_.shape)
#print(linearsvmclf.coef_)
linearsvmclf_coef_array= {}
linearsvmclf_coef_array[column]= linearsvmclf.coef_
TopTenFeat = sorted(linearsvmclf_coef_array.items(), key=lambda x: x[1])[-11:-1]
for f in TopTenFeat:
    print(f)

#### iii. How do these results compare with your results from decision trees, above?

#TODO

### 3. Regression
#### a. Use LinearRegression to learn a linear model directly predicting the crime rate per capita (ViolentCrimesPerPop).
##### i. Using 10-fold cross-validation, what is the estimated meansquared- error (MSE) of the model?

In [658]:
Y= communities_crime_df['ViolentCrimesPerPop']
linearRegression = LinearRegression()
#df = df.drop('highCrime', 1)
mean_sq_err= cross_val_score(linearRegression,X, Y,None,scoring='neg_mean_squared_error',cv=10)
print(np.abs(mean_sq_err.mean()))

0.0200939693044


##### ii. What is the MSE on the training set (train on all the data then test on it all)?

In [659]:
linearRegression.fit(X,y)
print("Mean squared error: %.2f"
      % np.mean((linearRegression.predict(X) - Y) ** 2))

Mean squared error: 0.21


##### iii. What features are most predictive of a high crime rate? A low crime rate?

In [660]:
list_coef = np.array(linearRegression.coef_)
max_feature = np.where(list_coef == list_coef.max())
print("Most Predictive Feature of a High Crime Rate is", df.columns[np.argmax(list_coef)])
print("Most Predictive Feature of a Low Crime Rate is", df.columns[np.argmin(list_coef)])

Most Predictive Feature of a High Crime Rate is population
Most Predictive Feature of a Low Crime Rate is numbUrban


#### b.Now use Ridge regression to reduce the amount of overfitting, using RidgeCV to pick the best alpha from among (10, 1, 0.1, 0.01, and 0.001).
##### i. What is the estimated MSE of the model under 10-fold CV?

In [661]:
ridge_reg=linear_model.Ridge(alpha = 0.001)
ridge_reg.fit(df,Y)
print("Mean squared error: %.2f"
      % np.mean((ridge_reg.predict(df) - Y) ** 2))

Mean squared error: 0.02


##### ii. What is the MSE on the training set (train on all the data then test on it all)?

In [662]:
train_data, test_data = validation_curve(
    linear_model.Ridge (), X, Y, param_name="alpha", param_range=np.logspace(-3, 1, 10),
    cv=10, scoring="neg_mean_squared_error", n_jobs=1)
train_data_mean = np.mean(train_data, axis=1)
train_data_std = np.std(train_data, axis=1)
test_data_mean = np.mean(test_data, axis=1)
test_data_std = np.std(test_data, axis=1)


#### c. Now use polynomial features to do quadratic (second-order) polynomial regression.
##### i.What is the estimated MSE of the model under 10-fold CV?

In [663]:
poly_feature = PolynomialFeatures(degree  = 2,interaction_only = False)
poly_df = poly_feature.fit_transform(X)
ridge_reg.fit(poly_df,y)
print("Mean squared error: %.2f"
      % np.mean((ridge_reg.predict(poly_df) - Y) ** 2))

Mean squared error: 0.30


In [664]:
poly_df = pd.DataFrame(poly_df)
array = [[1,2],[2,3]]
poly = PolynomialFeatures(degree  = 2 )
array = poly.fit_transform(array)
poly_df.shape

## caluclating MSE
poly_mse= cross_val_score(linearRegression, poly_df, Y,None,scoring='neg_mean_squared_error',cv=10)
print(poly_mse.mean())

-0.129900108473


### 4. Dirty Data
#### Repeat the decision tree learning question for the full (non-clean) data set and present the results.
#### a. Are the CV results better or worse? What does this say about the effect of missing values?

In [665]:
# import dirty data
df_d = pd.read_csv('communities-crime-full.csv')
df_d.head()

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [666]:
# Making data clean 
#communities_crime_df.drop('ViolentCrimesPerPop', axis=1, inplace=True)
df_d_clean = df_d.replace('?', np.nan)
df_d_clean = df_d_clean.drop('community', axis=1).drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1).drop('county', axis=1)
cols_numeric = df_d_clean.columns
print(cols_numeric)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
Y = imp.fit_transform(df_d_clean)
#imputer = Imputer(missing_values = np.nan, strategy="mean", axis=0)
#df_full_clean = imputer.fit_transform(df_d_clean)
df_d_clean = pd.DataFrame(Y)
print(df_d_clean.columns)
df_d_clean.columns = cols_numeric
df_d_clean['highCrime'] = df_d_clean.apply(setHighCrime, axis=1)

#df_full_clean.groupby('highCrime').size() * 100 / len(df_full_clean)

Index(['population', 'householdsize', 'racepctblack', 'racePctWhite',
       'racePctAsian', 'racePctHisp', 'agePct12t21', 'agePct12t29',
       'agePct16t24', 'agePct65up',
       ...
       'LandArea', 'PopDens', 'PctUsePubTrans', 'PolicCars', 'PolicOperBudg',
       'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'LemasPctOfficDrugUn',
       'PolicBudgPerPop', 'ViolentCrimesPerPop'],
      dtype='object', length=123)
RangeIndex(start=0, stop=123, step=1)


In [667]:
y = df_d_clean["highCrime"]
X = df_d_clean.drop('highCrime', axis=1).drop('ViolentCrimesPerPop', axis=1)
#X
dt = DecisionTreeClassifier(max_depth= 3)

dt.fit(X,y)
predicted = dt.predict(X)

recall_score = metrics.recall_score(df_d_clean['highCrime'], predicted)
precision_score = metrics.precision_score(df_d_clean['highCrime'], predicted)
accuracy_score = metrics.accuracy_score(df_d_clean['highCrime'], predicted)

print("Training Accuracy = {} Precision = {} Recall = {}".format(accuracy_score,precision_score,recall_score))

Training Accuracy = 0.8360080240722166 Precision = 0.9003466204506065 Recall = 0.8305355715427658


#### GradientBoostingClassifier

In [670]:
df = communities_crime_df.drop('ViolentCrimesPerPop', axis=1).drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1).drop('highCrime', axis=1)
y = communities_crime_df["highCrime"]
X = df
clf = GradientBoostingClassifier()
clf.fit(X, y)
predicted = clf.predict(X)

recall_score = metrics.recall_score(communities_crime_df['highCrime'], predicted)
precision_score = metrics.precision_score(communities_crime_df['highCrime'], predicted)
accuracy_score = metrics.accuracy_score(communities_crime_df['highCrime'], predicted)

print("Training Accuracy = {} Precision = {} Recall = {}".format(accuracy_score,precision_score,recall_score))



Training Accuracy = 0.9257400903161064 Precision = 0.9408 Recall = 0.9408
